The datasets used here are taken from [this](https://github.com/Nilabhra/kolkata_nlp_workshop_2019) repository. 

In [1]:
import pandas as pd

train = pd.read_csv('https://raw.githubusercontent.com/Nilabhra/kolkata_nlp_workshop_2019/master/data/train.csv')
validation = pd.read_csv('https://raw.githubusercontent.com/Nilabhra/kolkata_nlp_workshop_2019/master/data/valid.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Nilabhra/kolkata_nlp_workshop_2019/master/data/test.csv')

In [2]:
train.shape, validation.shape, test.shape

((9131, 3), (1142, 3), (1141, 3))

In [6]:
train.head()

,text,class
0,"I ordered a biryani, and the taste of the Biry...",positive
1,A nice place to hangout since it has both the ...,positive
2,This place is awesome for having lunch or dinn...,positive
3,I got shell of egg in the egg roll. as a resul...,negative
4,"Their biryani is oily, with a bit disconcertin...",negative


In [7]:
validation.head()

,text,class
0,The food was excellent with surplus quantity. ...,positive
1,This place nearer to the Gitanjali metro stati...,positive
2,Ordered for Aloo tikki with choley just now @0...,negative
3,Hatari is one of those restaurants that our fa...,positive
4,Disappointing.......\nThey have altered the ta...,negative


In [8]:
test.head()

,text,class
0,This place is amazing. I think the best place ...,positive
1,This place has been on my list for quite some ...,positive
2,What a wonderful cold winter evening it was. M...,positive
3,BabBQ had always been a personal favorite when...,positive
4,Know for its Deep Dish Pizza this place is sur...,negative


In [9]:
train['text'].loc[0]

'I ordered a biryani, and the taste of the Biryani was beyond my expectations and the quantity was also enough comparatively to the price!\nReally nice much appreciable'

### Removing digits for the text

In [2]:
from string import digits

def remove_digits(s):
    remove_digits = str.maketrans('', '', digits)
    res = s.translate(remove_digits)
    return res

In [3]:
train['text'] = train['text'].apply(remove_digits)
validation['text'] = validation['text'].apply(remove_digits)

### Bag of words representation 

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words=None, lowercase=True,
                             ngram_range=(1, 1), min_df=2, binary=True)

train_features = vectorizer.fit_transform(train['text'])
train_labels = train['class']

valid_features = vectorizer.transform(validation['text'])
valid_labels = validation['class']

### Label encode the classes

In [5]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train_labels = le.fit_transform(train_labels)
valid_labels = le.transform(valid_labels)

### Model building and compilation

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dropout, Dense

In [7]:
model = keras.Sequential()

model.add(Dropout(rate=0.2, input_shape=train_features.shape[1:]))
for _ in range(2):
        model.add(Dense(units=64, activation='relu'))
        model.add(Dropout(rate=0.2))
model.add(Dense(units=1, activation='sigmoid'))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Colocations handled automatically by placer.


In [8]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

In [9]:
# Define an EarlyStopping callback
es_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

### We are ready to train the model and validate

In [10]:
model.fit(train_features,
                    train_labels,
                    epochs=15,
                    batch_size=512,
                    validation_data=(valid_features, valid_labels),
                    callbacks=[es_cb],
                    verbose=1)

Train on 9131 samples, validate on 1142 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/15
9131/9131 [==============================] - 2s 214us/sample - loss: 0.6426 - acc: 0.6267 - val_loss: 0.5817 - val_acc: 0.6471
Epoch 2/15
9131/9131 [==============================] - 2s 177us/sample - loss: 0.5433 - acc: 0.6884 - val_loss: 0.4887 - val_acc: 0.7907
Epoch 3/15
9131/9131 [==============================] - 2s 178us/sample - loss: 0.4450 - acc: 0.8112 - val_loss: 0.4183 - val_acc: 0.8240
Epoch 4/15
9131/9131 [==============================] - 2s 208us/sample - loss: 0.3724 - acc: 0.8431 - val_loss: 0.4045 - val_acc: 0.8205
Epoch 5/15
9131/9131 [==============================] - 2s 195us/sample - loss: 0.3157 - acc: 0.8723 - val_loss: 0.4085 - val_acc: 0.8275
Epoch 6/15
9131/9131 [==============================] - 2s 165us/sample - loss: 0.2845 - acc: 0.8803 - val_loss: 0.4265 - val_acc: 0.8231
Epoch 7/15
9131/9131 [==============================] - 2s 176us/sample - lo

### How good is the model? 

In [11]:
test['text'] = test['text'].apply(remove_digits)
test_features = vectorizer.transform(test['text'])
test_labels = le.transform(test['class'])

In [12]:
results = model.evaluate(test_features, test_labels)
print("Accuracy: {0:.2f}%".format(results[1]*100.))

1141/1141 [==============================] - 0s 135us/sample - loss: 0.5658 - acc: 0.7914
Accuracy: 79.14%


### Combining the training and validation sets and retraining the model

In [13]:
data = pd.concat((train, validation), axis=0)

vectorizer = CountVectorizer(stop_words=None, lowercase=True,
                             ngram_range=(1, 1), min_df=2)

features = vectorizer.fit_transform(data['text'])
labels = le.fit_transform(data['class'])

test_features = vectorizer.transform(test['text'])
test_labels = le.transform(test['class'])

In [14]:
model = keras.Sequential()

model.add(Dropout(rate=0.2, input_shape=features.shape[1:]))
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

In [15]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

In [16]:
model.fit(features,
                    labels,
                    epochs=15,
                    batch_size=512,
                    validation_data=(test_features, test_labels),
                    callbacks=[es_cb],
                    verbose=1)

Train on 10273 samples, validate on 1141 samples
Epoch 1/15
10273/10273 [==============================] - 2s 215us/sample - loss: 0.6408 - acc: 0.6189 - val_loss: 0.5679 - val_acc: 0.6670
Epoch 2/15
10273/10273 [==============================] - 2s 201us/sample - loss: 0.5159 - acc: 0.7512 - val_loss: 0.4445 - val_acc: 0.8168
Epoch 3/15
10273/10273 [==============================] - 2s 201us/sample - loss: 0.4118 - acc: 0.8279 - val_loss: 0.4257 - val_acc: 0.8195
Epoch 4/15
10273/10273 [==============================] - 2s 211us/sample - loss: 0.3570 - acc: 0.8544 - val_loss: 0.4232 - val_acc: 0.8317
Epoch 5/15
10273/10273 [==============================] - 2s 220us/sample - loss: 0.3127 - acc: 0.8749 - val_loss: 0.4362 - val_acc: 0.8265
Epoch 6/15
10273/10273 [==============================] - 2s 175us/sample - loss: 0.2766 - acc: 0.8925 - val_loss: 0.4824 - val_acc: 0.8177
Epoch 7/15
10273/10273 [==============================] - 2s 173us/sample - loss: 0.2463 - acc: 0.9021 - val_lo

> We will use this model for serving. 

### Creating `sklearn` pipeline for deployment

For this we will have to wrap the `tf-keras` model into a `scikit-learn` compatible model class. Then we can use that as a part of a `scikit-learn` pipeline. Let's start by defining a `create_model()` method which would be required for the _scikit-learn model wrapping_ part. 

In [17]:
def create_model():
    model = keras.Sequential()

    model.add(Dropout(rate=0.2, input_shape=(9270,)))
    model.add(Dense(units=64, activation='relu'))
    model.add(Dropout(rate=0.2))
    model.add(Dense(units=64, activation='relu'))
    model.add(Dropout(rate=0.2))
    model.add(Dense(units=1, activation='sigmoid'))

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['acc'])
    
    return model

In [32]:
from keras.wrappers.scikit_learn import KerasClassifier

# Same epoch and same batch size
model = KerasClassifier(build_fn=create_model, epochs=15, batch_size=512, verbose=0)

In [34]:
# Construct the pipeline
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('feature_transformer', vectorizer),
                  ('classifier', model)])

In [35]:
# Fit the pipeline
pipeline.fit(data['text'], labels)

Pipeline(memory=None,
     steps=[('feature_transformer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=2,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('classifier', <keras.wrappers.scikit_learn.KerasClassifier object at 0x127bfa278>)])

In [38]:
# Use the pipeline to make inferences
le.inverse_transform(pipeline.predict([remove_digits('I had a very bad experience you know.')]))

/miniconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:273: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array(['negative'], dtype=object)

In [40]:
# Ready to serialize/pickle the model
from sklearn.externals import joblib

# Courtesy: https://bit.ly/2IwQKSS
# Save the Keras model first
pipeline.named_steps['classifier'].model.save('model/keras_model.h5')

# This hack allows us to save the sklearn pipeline
pipeline.named_steps['classifier'].model = None

# Finally, save the pipeline
joblib.dump(pipeline, 'model/sklearn_pipeline.pkl')

['model/sklearn_pipeline.pkl']

In [46]:
# Load the pipeline first
pipeline = joblib.load('model/sklearn_pipeline.pkl')

# Then, load the Keras model
from keras.models import load_model
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform

with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
        pipeline.named_steps['classifier'].model = load_model('model/keras_model.h5')

# Start making inference
le.inverse_transform(pipeline.predict([remove_digits('I had a very bad experience you know.')]))[0]

'negative'